In [ ]:
import os
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
os.environ['HF_TOKEN'] = hf_token

In [ ]:
!pip install transformers==4.48.0 peft==0.14.0 trl==0.12.0 bitsandbytes==0.45.3 accelerate==1.2.1

In [ ]:
# Quantization config
from transformers import BitsAndBytesConfig
import torch

quantization_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

In [ ]:
# PEFT(Parameter Efficient Fine Tuning) - Setting LoRA
from peft import LoraConfig
peft_config = LoraConfig(
    lora_alpha=16, # scaling factor applied to BA when updating weights
    lora_dropout=0,
    r=16, # rank of the low-ranked matrices
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","v_proj","k_proj","o_proj","gate_proj","up_proj","down_proj"]
)

In [ ]:
# Load a base Llama model
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",
    quantization_config = quantization_config, # A base model quantized to 4 bit - cannot train this base model
    device_map = {"": 0}                       # Instead, we can fine-tune this with QLoRA.
)

In [ ]:
# Set Tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B") # Each model has its own tokenizer
tokenizer.add_special_tokens({"pad_token": "<|reserved_special_token_250|>"}) # Ensure the special token has consistent length
model.config.pad_token_id = tokenizer.pad_token_id


In [ ]:
# Instruction Formatting (Prompt/Response Format) - very important !!
# When you train an LLM, the instruction format you use in the training data should be the same
# as the format you use when asking the model questions later.
EOS_TOKEN = tokenizer.eos_token

def convert_to_alpaca_format(instruction, response):
    alpaca_format_str = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.\
    \n\n### Instruction:\n{instruction}\n\n### Response:\n{response}\
    """

    return alpaca_format_str
''' (Sample instruction format below)
Below is an insruction that describe a task. Write a response that appropriately completes the request.

### Instruction:
What is LLM

### Response:
LLM is an AI model. <--(Before output is generated, this line is empty)
'''

def prompt_formatting_func(examples):
    instructions = examples["instruction"]
    outputs      = examples["output"]
    texts = [] # Adding EOS_TOKEN is very important for a model to learn when to stop generating text
    for instruction, output in zip(instructions, outputs): # Create a single tuple per iteration by zip
        alpaca_formatted_str = convert_to_alpaca_format(instruction, output) + EOS_TOKEN # Don't forget adding end of text token!!
        texts.append(alpaca_formatted_str)
    return { "text" : texts, }

In [ ]:
# Dataset Load
from datasets import load_dataset

dataset = load_dataset("yahma/alpaca-cleaned", split="train")

dataset = dataset.shuffle(seed=42) # Ensure fine-tuning result is reserved
no_input_dataset = dataset.filter(lambda example: example['input'] == '') # A bit different from typical filter() in Python 3. Data parameter is skipped
                                                                          # 'input' column is not needed. We use only 'instruction' and 'output' column
mapped_dataset = no_input_dataset.map(prompt_formatting_func, batched=True) # Get a map object from 'prompt_formatting_func'
split_dataset = mapped_dataset.train_test_split(test_size=0.01, seed=42)

train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

In [ ]:
# Data Collator
# 1) Apply tokenizer for padding token
# 2) ** set response template for a model to learn how to answer (how to generate text) **
from trl import DataCollatorForCompletionOnlyLM
data_collator_param = {}
response_template = "### Response:\n" # 2) The part that follows this 'Response' is where the weights should be updated through the loss function
collator = DataCollatorForCompletionOnlyLM(response_template=response_template, tokenizer=tokenizer, mlm=False)
data_collator_param["data_collator"] = collator

In [ ]:
# local output dir
local_output_dir = "/content/fine_tune_output"

In [ ]:
!mkdir {local_output_dir}

In [ ]:
# tensorboard
%load_ext tensorboard
%tensorboard --logdir '{local_output_dir}/runs'

In [ ]:
# Training setup
from trl import SFTTrainer
from transformers import TrainingArguments

training_arguments = TrainingArguments(
  output_dir=local_output_dir,
  report_to = "tensorboard",
  per_device_train_batch_size = 2,
  per_device_eval_batch_size = 2,
  gradient_accumulation_steps = 8,
  warmup_steps = 50,
  max_steps = 100, # Total number of training samples = 2 (samples per batch) * 50 (max steps)
  eval_steps=10,   # The weights will be updated 100 times (due to max_steps == 100)
  save_steps=50,
  evaluation_strategy="steps",
  save_strategy="steps",
  learning_rate = 1e-4,
  logging_steps = 1,
  optim = "adamw_8bit",
  weight_decay = 0.01,
  lr_scheduler_type = "constant_with_warmup",
  seed = 42,
  gradient_checkpointing = True,
  gradient_checkpointing_kwargs={'use_reentrant':True}
)

trainer = SFTTrainer( # Trainer optimized for supervised fine-tuning
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
     peft_config=peft_config, # Depending on trainer, either 1) Both base model and peft model should be provided // OR 2) Only peft model is needed
    dataset_text_field = "text", # prompt text for training
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = training_arguments,
    **data_collator_param
)

In [ ]:
train_stats = trainer.train()

In [ ]:
# Copy to Google drive
from google.colab import drive
drive.mount('/content/drive')

!ls /content/fine_tune_output

In [ ]:
!cp -r {local_output_dir} /content/drive/MyDrive

In [ ]:
!ls /content/fine_tune_output

In [ ]:
from google.colab import drive
drive.mount('/content/drive')